In [1]:
import kagglehub
import numpy as np
import pandas as pd

from tabulate import tabulate
import matplotlib.pyplot as plt

from wordcloud import WordCloud
import seaborn as sns
from textblob import TextBlob

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

import string
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('wordnet')

c:\Users\nikhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Preprocessing

In [2]:
# Download latest version
path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

print("Path to dataset files:", path)

# load datasets
df=pd.read_csv(path+'/WELFake_Dataset.csv')
df.head()

Path to dataset files: C:\Users\nikhi\.cache\kagglehub\datasets\saurabhshahane\fake-news-classification\versions\77


,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [3]:
df.shape

(72134, 4)

In [4]:
df['article'] = df['title'] + df['text']
df = df[['article', 'label']]
df.head()

,article,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
1,NaN,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",0
4,SATAN 2: Russia unvelis an image of its terrif...,1


In [5]:
df.dropna(inplace=True)
df.drop_duplicates(subset='article', inplace=True)
df.shape

(63121, 2)

In [6]:
df['sentiment_polarity'] = df['article'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
df['sentiment_subjectivity'] = df['article'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity)

In [7]:
df.head()

,article,label,sentiment_polarity,sentiment_subjectivity
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,0.034641,0.296824
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,0.190402,0.512798
3,"Bobby Jindal, raised Hindu, uses story of Chri...",0,0.111493,0.387106
4,SATAN 2: Russia unvelis an image of its terrif...,1,0.013342,0.403577
5,About Time! Christian Group Sues Amazon and SP...,1,-0.100461,0.428571


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['article'], df['label'], test_size=0.2, random_state=42, stratify=df['label'])
overlap = set(X_train).intersection(set(X_test))
print(f"Number of overlapping samples between train and test sets: {len(overlap)}")

Number of overlapping samples between train and test sets: 0


In [9]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    text = re.sub(r'\b([a-zA-Z])\.(?=[a-zA-Z])', r'\1specialdot', text)  # Preserve acronyms
    text = re.sub(r'\b([a-zA-Z])\.(?=\s|$)', r'\1specialdot', text)
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.replace("specialdot", ".")
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize, remove stopwords, and lemmatize
    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    stop_words.discard("not")
    stop_words.discard("no")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Vectorize text
def vectorize(text, maxFeats, ngram):
    vectorizer = TfidfVectorizer(max_features=maxFeats, ngram_range=ngram, stop_words='english')
    X_tfidf = vectorizer.fit_transform(text)
    return X_tfidf, vectorizer

# Load data
X_train, X_test, y_train, y_test = train_test_split(
    df['article'], df['label'], 
    test_size=0.2, random_state=42, stratify=df['label']
)

# Preprocess data
X_train_processed = X_train.apply(preprocess_text)
X_test_processed = X_test.apply(preprocess_text)

# Vectorize data
X_train_vectorized, vectorizer = vectorize(X_train_processed, 2000, (1,1))
X_test_vectorized = vectorizer.transform(X_test_processed)

# Train model
#model = RandomForestClassifier()
#model.fit(X_train_vectorized, y_train)

# Predict and evaluate on test set
#y_pred = model.predict(X_test_vectorized)
#print(classification_report(y_test, y_pred))
#print(f"Test Set Accuracy: {accuracy_score(y_test, y_pred)}")

# # Cross-validation
# scores = cross_val_score(model, X_train_vectorized, y_train, cv=5, scoring='accuracy')
# print("Cross-validation Accuracies:", scores)
# print("Average Accuracy:", scores.mean())


In [10]:
X_train_vectorized

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5681473 stored elements and shape (50496, 2000)>

# Train SVM Model

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

def run_svm_pipeline(X_train, X_test, y_train, y_test):
    """
    Implements an SVM classifier for fake news detection
    """
    # Initialize SVM with a linear kernel
    svm_model = SVC(kernel='linear', probability=True, random_state=42)
    
    # Train model on the training data
    svm_model.fit(X_train, y_train)
    
    # Predict on test set
    y_pred = svm_model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    return svm_model, accuracy, report

# Run the SVM pipeline
model, accuracy, report = run_svm_pipeline(X_train_vectorized, X_test_vectorized, y_train, y_test)

# Display accuracy and report:
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", report)


Accuracy: 93.67%
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94      6959
           1       0.93      0.93      0.93      5666

    accuracy                           0.94     12625
   macro avg       0.94      0.94      0.94     12625
weighted avg       0.94      0.94      0.94     12625



In [ ]:
# Cross-validation with GridSearchCV
from sklearn.model_selection import GridSearchCV

def run_svm_pipeline_cv(X_train, y_train):
    """
    Implements an SVM classifier for fake news detection
    """
    svm_model = SVC(kernel='linear', probability=True, random_state=42)
    
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01, 0.001],
        'kernel': ['linear', 'rbf', 'poly']
    }
    
    grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
    
    grid_search.fit(X_train, y_train)
    
    return grid_search

grid_search = run_svm_pipeline_cv(X_train_vectorized, y_train)

# Test SVM Model

In [13]:
# Download latest version
path = kagglehub.dataset_download("bhavikjikadara/fake-news-detection")

print("Path to dataset files:", path)

# load datasets
fake=pd.read_csv(path+'/fake.csv')
fake['label'] = 0
true=pd.read_csv(path+'/true.csv')
true['label'] = 1
test = pd.concat([fake, true])
test.head()

100%|██████████| 41.0M/41.0M [00:04<00:00, 10.5MB/s]

Extracting files...


Path to dataset files: C:\Users\nikhi\.cache\kagglehub\datasets\bhavikjikadara\fake-news-detection\versions\1


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [14]:
test['article'] = test['title'] + test['text']
test = test[['article', 'label']]

In [15]:
test.dropna(inplace=True)
test.drop_duplicates(subset='article', inplace=True)
test.shape

(39105, 2)

In [16]:
test['sentiment_polarity'] = test['article'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
test['sentiment_subjectivity'] = test['article'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity)
test.head()

,article,label,sentiment_polarity,sentiment_subjectivity
0,Donald Trump Sends Out Embarrassing New Year’...,0,0.083136,0.597204
1,Drunk Bragging Trump Staffer Started Russian ...,0,-0.020811,0.344802
2,Sheriff David Clarke Becomes An Internet Joke...,0,-0.012345,0.541969
3,Trump Is So Obsessed He Even Has Obama’s Name...,0,-0.038021,0.413021
4,Pope Francis Just Called Out Donald Trump Dur...,0,-0.011722,0.495222


In [17]:
test['label'] = 1 - test['label']

In [18]:
test.head()

,article,label,sentiment_polarity,sentiment_subjectivity
0,Donald Trump Sends Out Embarrassing New Year’...,1,0.083136,0.597204
1,Drunk Bragging Trump Staffer Started Russian ...,1,-0.020811,0.344802
2,Sheriff David Clarke Becomes An Internet Joke...,1,-0.012345,0.541969
3,Trump Is So Obsessed He Even Has Obama’s Name...,1,-0.038021,0.413021
4,Pope Francis Just Called Out Donald Trump Dur...,1,-0.011722,0.495222


In [ ]:
X, y = test['article'], test['label']
X = X.apply(preprocess_text)
X = vectorizer.transform(X)

In [20]:
# Evaluate the SVM model
y_pred = model.predict(X)
accuracy = (y_pred == y).mean()

print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9738
